In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sys
import time

In [2]:
attr = {'Goalkeeping': 'stats_keeper_', 'Goalkeeping_adv': 'stats_keeper_adv_','Defense': 'stats_defense_',
        'Shooting': 'stats_shooting_','Passing': 'stats_passing_','Pass_types': 'stats_passing_types_',
        'GCA': 'stats_gca_','Possession': 'stats_possession_','Playing_time': 'stats_playing_time_','Misc': 'stats_misc_'}

In [3]:
Leagues = {
    'Premier League'  :{'id':9,'link':'https://fbref.com/en/comps/9/Premier-League-Stats','league':'Premier League',
                        'main_table':'results2023-202491_overall','season':'2023-2024','country':'England'},#Done
    
    'EFL'             :{'id':10,'link':'https://fbref.com/en/comps/10/Championship-Stats','league':'EFL',
                        'main_table':'results2023-2024101_overall','season':'2023-2024','country':'England'},#Done
    
    'Serie A'         :{'id':11,'link':'https://fbref.com/en/comps/11/Serie-A-Stats','league':'Serie A',
                        'main_table':'results2023-2024111_overall','season':'2023-2024','country':'Italy'},#Done
    
    'La Liga'         :{'id':12,'link':'https://fbref.com/en/comps/12/La-Liga-Stats','league':'La Liga',
                        'main_table':'results2023-2024121_overall','season':'2023-2024','country':'Spain'},#Done
    
    'Ligue 1'         :{'id':13,'link':'https://fbref.com/en/comps/13/Ligue-1-Stats','league':'Ligue 1',
                        'main_table':'results2023-2024131_overall','season':'2023-2024','country':'France'},#Done
    
    'Segunda Division':{'id':17,'link':'https://fbref.com/en/comps/17/Segunda-Division-Stats','league':'Segunda Division',
                        'main_table':'results2023-2024171_overall','season':'2023-2024','country':'Spain'},#Done
    
    'Serie B'         :{'id':18,'link':'https://fbref.com/en/comps/18/Serie-B-Stats','league':'Serie B',
                        'main_table':'results2023-2024181_overall','season':'2023-2024','country':'Italy'},#Done
    
    'Bundesliga'      :{'id':20,'link':'https://fbref.com/en/comps/20/Bundesliga-Stats','league':'Bundesliga',
                        'main_table':'results2023-2024201_overall','season':'2023-2024','country':'Germany'},#Done
    
    'Primera Division':{'id':21,'link':'https://fbref.com/en/comps/21/2023/2023-Primera-Division-Stats','league':'Primera Division',
                        'main_table':'results2023211_overall','season':'2023','country':'Argentina'},#Done
    
    'MLS_east'        :{'id':22,'link':'https://fbref.com/en/comps/22/2023/2023-Major-League-Soccer-Stats','league':'MLS',
                        'main_table':'results2023221Eastern-Conference_overall','season':'2023','country':'USA'},#Done
    
    'MLS_west'        :{'id':22,'link':'https://fbref.com/en/comps/22/2023/2023-Major-League-Soccer-Stats','league':'MLS',
                        'main_table':'results2023221Western-Conference_overall','season':'2023','country':'USA'},#Done
    
    'Eredivisie'      :{'id':23,'link':'https://fbref.com/en/comps/23/Eredivisie-Stats','league':'Eredivisie',
                        'main_table':'results2023-2024231_overall','season':'2023-2024','country':'Netherlands'},#Done
    
    'Brazilian League':{'id':24,'link':'https://fbref.com/en/comps/24/2023/2023-Serie-A-Stats','league':'Brazilian League',
                        'main_table':'results2023241_overall','season':'2023','country':'Brazil'},#Done
    
    'Super Lig'       :{'id':26,'link':'https://fbref.com/en/comps/26/Super-Lig-Stats','league':'Super Lig',
                        'main_table':'results2023-2024261_overall','season':'2023-2024','country':'Turkey'},#Done
    
    'Liga MX'         :{'id':31,'link':'https://fbref.com/en/comps/31/Liga-MX-Stats','league':'Liga MX',
                        'main_table':'results2023-2024310_overall','season':'2023-2024','country':'Mexico'},#Done
    
    'Premeira Liga'   :{'id':32,'link':'https://fbref.com/en/comps/32/Primeira-Liga-Stats','league':'Primeira Liga',
                        'main_table':'results2023-2024321_overall','season':'2023-2024','country':'Portugal'},#Done
    
    '2. Bundesliga'   :{'id':33,'link':'https://fbref.com/en/comps/33/2-Bundesliga-Stats','league':'2. Bundesliga',
                        'main_table':'results2023-2024331_overall','season':'2023-2024','country':'Germany'},#Done
    
    'Pro League'      :{'id':37,'link':'https://fbref.com/en/comps/37/Belgian-Pro-League-Stats','league':'Pro League',
                        'main_table':'results2023-2024370_overall','season':'2023-2024','country':'Belgium'},#Done
    
    'Ligue 2'         :{'id':60,'link':'https://fbref.com/en/comps/60/Ligue-2-Stats','league':'Ligue 2',
                        'main_table':'results2023-2024601_overall','season':'2023-2024','country':'France'}#Done
}

In [4]:
def scrap_clubs_links(league):
    Clubs = {}
    response = requests.get(league['link'])
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.select(f'#{league["main_table"]} tbody')
        if table:
            table_html = str(table[0])
            table_soup = BeautifulSoup(table_html, 'html.parser')
            clubs = table_soup.find_all('tr')
            for row in clubs:
                Club = row.find('td', {'data-stat': 'team'}).text.strip()
                link = 'https://fbref.com/' + row.find('a')['href']
                Clubs[Club] = link
        else:
            print("Table not found on the page.")
    else:
        print("Failed to retrieve data. Status code:", response.status_code)
    return Clubs

In [5]:
def Scrap (table, club,keys,league):
    if table:
        table_html = str(table[0])
        table_soup = BeautifulSoup(table_html, 'html.parser')
        player_rows = table_soup.find_all('tr')
        player_data = []
        for row in player_rows:
            player_dict={}
            for key in keys:
                if key == 'club':
                    player_dict[key] = club
                elif key == 'league':
                    player_dict[key] = league
                elif key == 'player':
                    player_dict[key] = row.find('th', {'data-stat': key}).text.strip()
                elif key == 'nationality':
                    player_dict[key] = row.find('td', {'data-stat': key}).text.strip()[-3:]
                else:
                    player_dict[key] = row.find('td', {'data-stat': key}).text.strip()
            player_data.append(player_dict)
        df = pd.DataFrame(player_data)
    else:
        print("Table not found on the page.")
    return df

In [6]:
def Scrap_ds(response,j):
    soup = BeautifulSoup(response.text,'html.parser')
    table = soup.select(f'table#{j} tbody')
    table_html = str(table[0])
    table_soup = BeautifulSoup(table_html, 'html.parser')
    ds = table_soup.find_all('tr')
    data_stat_values = [tag.get('data-stat') for tag in ds[0].find_all(['td', 'th'])]
    data_stat_values.insert(4, 'club')
    data_stat_values.insert(5, 'league')
    data_stat_values.pop()
    return data_stat_values

In [7]:
def Scrap_data(league):
    Clubs = scrap_clubs_links(league)
    data = {i: [] for i in attr.keys()}
    ds = {att: None for att in attr.keys()}  # Initialize ds with None values
    
    # Initialize ds outside the main loop
    for att, st_id in attr.items():
        ds[att] = Scrap_ds(requests.get(list(Clubs.values())[0]), st_id + str(league['id']))
    
    cpt = 1
    for club, link in Clubs.items():
        response = requests.get(link)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            for att, st_id in attr.items():
                table = soup.select(f'table#{st_id+str(league["id"])} tbody')
                data[att].append(Scrap(table, club, ds[att], league['league']))
            print(f'{cpt} - {club} data scraped successfully')
            cpt += 1
            time.sleep(7)
        else:
            print('status code : ', response.status_code)
            break
    return data


In [8]:
Data = Scrap_data(Leagues['EFL'])

1 - Leicester City data scraped successfully
2 - Ipswich Town data scraped successfully
3 - Leeds United data scraped successfully
4 - Southampton data scraped successfully
5 - West Brom data scraped successfully
6 - Norwich City data scraped successfully
7 - Hull City data scraped successfully
8 - Middlesbrough data scraped successfully
9 - Coventry City data scraped successfully
10 - Preston data scraped successfully
11 - Bristol City data scraped successfully
12 - Cardiff City data scraped successfully
13 - Millwall data scraped successfully
14 - Swansea City data scraped successfully
15 - Watford data scraped successfully
16 - Sunderland data scraped successfully
17 - Stoke City data scraped successfully
18 - QPR data scraped successfully
19 - Blackburn data scraped successfully
20 - Sheffield Weds data scraped successfully
21 - Plymouth Argyle data scraped successfully
22 - Birmingham City data scraped successfully
23 - Huddersfield data scraped successfully
24 - Rotherham Utd dat

In [10]:
Data['Goalkeeping'][0].head()

,player,nationality,position,age,club,league,gk_games,gk_games_starts,gk_minutes,minutes_90s,...,gk_wins,gk_ties,gk_losses,gk_clean_sheets,gk_clean_sheets_pct,gk_pens_att,gk_pens_allowed,gk_pens_saved,gk_pens_missed,gk_pens_save_pct
0,Mads Hermansen,DEN,GK,23,Leicester City,EFL,44,44,"3,960",44.0,...,29,4,11,13,29.5,1,0,1,0,100.0
1,Jakub Stolarczyk,POL,GK,22,Leicester City,EFL,2,2,180,2.0,...,2,0,0,2,100.0,0,0,0,0,


In [11]:
# Dictionary that will host the concatenated dataframes
total_data = {}

In [12]:
for i,j in Data.items():
    total_data[i] = pd.concat(j, ignore_index=True)

In [13]:
# Checking for dupliates
duplicates = total_data['Passing']['player'].duplicated()

if duplicates.any():
    print("The column has duplicate values. They are:")
    print(total_data['Passing'][duplicates]['player'])
else:
    print("The column does not have any duplicate values.")

The column has duplicate values. They are:
138          Jeremy Sarmiento
219             Sam Greenwood
224               Luke Ayling
312    Taylor Gardner-Hickman
318           Andreas Weimann
319               Scott Twine
406       Przemysław Płacheta
410                Joël Piroe
459           Leo Fuhr Hjelde
486              Lynden Gooch
508                Josh Tymon
559              Lewis Travis
560            Kyle McFadzean
566               Niall Ennis
567               Yasin Ayari
589                  Ike Ugbo
596          Ian Carlo Poveda
598              John Buckley
604         Kristian Pedersen
616                 Finn Azaz
622               Luke Cundle
626               Darko Gyabi
628              Adam Forshaw
647               Cody Drameh
658             Andre Dozzell
660            Alex Pritchard
689               Rhys Healey
708         Cameron Humphreys
715            Andy Rinomhota
728                 Ben Wiles
Name: player, dtype: object


In [14]:
#These duplicates are for players' transfers during the season
total_data['Passing'][total_data['Passing']['player']=='Jeremy Sarmiento']

,player,nationality,position,age,club,league,minutes_90s,passes_completed,passes,passes_pct,...,passes_pct_long,assists,xg_assist,pass_xa,xg_assist_net,assisted_shots,passes_into_final_third,passes_into_penalty_area,crosses_into_penalty_area,progressive_passes
46,Jeremy Sarmiento,ECU,FW,21,Ipswich Town,EFL,7.1,124,170,72.9,...,64.7,1,1.4,0.9,-0.4,13,14,5,1,22
138,Jeremy Sarmiento,ECU,"FW,MF",21,West Brom,EFL,8.1,168,230,73.0,...,53.3,0,0.7,0.9,-0.7,7,11,11,4,17


In [15]:
# Exporting the scrapped data into csv files
for i in total_data.keys():
    total_data[i].to_csv(i+"_EFL_23_24.csv", index=False)

In [26]:
import os
import glob
#To empty the returned files

files = glob.glob('/kaggle/working/*')
for f in files:
    os.remove(f)